In [1]:
from google.colab import drive
drive.mount('/content/drive/')

!cp "/content/drive/My Drive/Dissertation/envs/point_push.py" .

Mounted at /content/drive/


In [2]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_push/{name}" 

    torch.save({
      'controller': {
          'critic': model.critic.state_dict(),
          'actor': model.actor.state_dict(),
      }
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_push/{name}" 
    checkpoint = torch.load(path)

    model.critic.load_state_dict(checkpoint['controller']['critic'])
    model.critic_target = copy.deepcopy(model.critic)
    
    model.actor.load_state_dict(checkpoint['controller']['actor'])
    model.actor_target = copy.deepcopy(model.actor)

    model.eval()

In [3]:
%matplotlib inline

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [4]:
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)

In [5]:
from point_push import PointPushEnv 
env = NormalizedEnv(PointPushEnv(4))

***

In [6]:
def plot_durations(episode_durations, actions):
    fig, axs = plt.subplots(2, figsize=(10,10))
    
    durations_t, durations = list(map(list, zip(*episode_durations)))
    durations = torch.tensor(durations, dtype=torch.float)
    
    fig.suptitle('Training')
    axs[0].set_xlabel('Episode')
    axs[0].set_ylabel('Reward')
    
    axs[0].plot(durations_t, durations.numpy())

    durations_t, durations = list(map(list, zip(*actions)))
    durations = torch.tensor(durations, dtype=torch.float)
    axs[1].plot(durations_t, durations.numpy())
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [7]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [8]:
def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [9]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

***

In [10]:
class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, 128)
        self.fc2 = nn.Linear(128, 128)
        self.head = nn.Linear(128, nb_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.head(x))

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(nb_states + nb_actions, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 1)

        # Q2 architecture
        self.l4 = nn.Linear(nb_states + nb_actions, 128)
        self.l5 = nn.Linear(128, 128)
        self.l6 = nn.Linear(128, 1)
    
    def forward(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [11]:
BATCH_SIZE = 64
GAMMA = 0.99

# https://spinningup.openai.com/en/latest/algorithms/td3.html
class TD3(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(TD3, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        
        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=0.0001)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optimizer  = optim.Adam(self.critic.parameters(), lr=0.0001)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)
        
        #Create replay buffer
        self.memory = ReplayMemory(100000)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=0.15, mu=0.0, sigma=0.2)

        # Hyper-parameters
        self.tau = 0.005
        self.depsilon = 1.0 / 10000
        self.policy_noise=0.2
        self.noise_clip=0.5
        self.policy_freq=2
        self.total_it = 0

        # 
        self.epsilon = 1.0
        self.is_training = True

    def update_policy(self):
        if len(self.memory) < BATCH_SIZE:
            return

        self.total_it += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)

        # Target Policy Smoothing
        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action_batch) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip).float()
            
            next_action = (
                self.actor_target(next_state_batch) + noise
            ).clamp(-1.0, 1.0).float()

            # Compute the target Q value
            # Clipped Double-Q Learning
            target_Q1, target_Q2 = self.critic_target(next_state_batch, next_action)
            target_Q = torch.min(target_Q1, target_Q2).squeeze(1)
            target_Q = (reward_batch + GAMMA * not_done_mask  * target_Q).float()
        
        # Critic update
        current_Q1, current_Q2 = self.critic(state_batch, action_batch)
      
        critic_loss = F.mse_loss(current_Q1, target_Q.unsqueeze(1)) + F.mse_loss(current_Q2, target_Q.unsqueeze(1))

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic.Q1(state_batch, self.actor(state_batch)).mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Target update
            soft_update(self.actor_target, self.actor, self.tau)
            soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def observe(self, s_t, a_t, s_t1, r_t, done):
        self.memory.store(s_t, a_t, s_t1, r_t, done)

    def random_action(self):
        return torch.tensor([np.random.uniform(-1.,1.,self.nb_actions)], device=device, dtype=torch.float)

    def select_action(self, s_t, warmup, decay_epsilon):
        if warmup:
            return self.random_action()

        with torch.no_grad():
            action = self.actor(s_t).squeeze(0)
            #action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * self.random_process.sample()).to(device).float()
            action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * np.random.uniform(-1.,1.,1)).to(device).float()
            action = torch.clamp(action, -1., 1.)

            action = action.unsqueeze(0)
            
            if decay_epsilon:
                self.epsilon -= self.depsilon
            
            return action

In [ ]:
import time
SAVE_OFFSET = 10

def train_model():
    global SAVE_OFFSET

    n_observations = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]
    
    agent = TD3(n_observations, n_actions).to(device)
    
    max_episode_length = 500
    
    agent.is_training = True
    episode_reward = 0.
    observation = None
    
    warmup = 100
    num_episodes = 4000 # M
    episode_durations = []
    actions = [(0,0)]

    steps = 0
    for i_episode in range(num_episodes):
        observation = env.reset()
        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        
        overall_reward = 0
        episode_steps = 0
        done = False
        while not done:        
            # agent pick action ...
            action = agent.select_action(state, i_episode <= warmup, True)

            # env response with next_observation, reward, terminate_info
            action_i = action.detach().cpu().squeeze(0).numpy()

            observation, reward, done, info = env.step(action_i)
            steps += 1

            #actions.append((steps, action_i[0]))
                
            if max_episode_length and episode_steps >= max_episode_length - 1:
                done = True
            episode_steps += 1 
                
            extrinsic_reward = torch.tensor([reward], device=device)
            
            overall_reward += reward
            
            next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            # agent observe and update policy
            agent.observe(state, action, next_state, extrinsic_reward, done)           
            state = next_state
            
            if i_episode > warmup:
                agent.update_policy()

        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations, actions)

        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if i_episode % 100 == 0:
                print(f"{i_episode}: {np.mean(dur[-100:])}")
            if i_episode >= 300 and i_episode % 100 == 0 and np.mean(dur[-100:]) <= -49.0:
                print(f"Unlucky after {i_episode} eps! Terminating...")
                return None
            if np.mean(dur[-100:]) >= 90:
                print(f"Solved after {i_episode} episodes!")
                save_model(agent, f"td3_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return agent
    return None

In [ ]:
i = 0
while i < 17:
    agent = train_model()
    
    if agent is not None:
        print(i)
        i += 1

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def eval_model(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for noise in np.arange(0,0.51,0.05):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                state = state + state_range * torch.FloatTensor(state.shape).uniform_(-noise, noise).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(noise, 2)].append(overall_reward / num_episodes)

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data + eps * sign_data_grad * state_range

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), state_min)

    return clipped_perturbed_data

def fgsm_action(state, agent, eps, target, targeted):
    state = state.clone().detach().requires_grad_(True)

    if targeted:
        # initial forward pass
        action = agent.actor(state)
        action = torch.clamp(action, -1., 1.)

        loss = F.mse_loss(action, target)
    else:
        loss = agent.critic.Q1(state, agent.actor(state)).mean()

    agent.actor.zero_grad()
    agent.critic.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad).float()
    return agent.select_action(state_p, False, False)

def apply_fgsm(agent, episode_durations, targeted):
    TARGET_ACTION = torch.tensor([[0.0, 0.0]], device=device, dtype=torch.float)

    agent.eval()

    max_episode_length = 500
    agent.is_training = False

    num_episodes = 100

    for eps in np.arange(0.0, 0.201, 0.02):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                action = fgsm_action(state, agent, eps, TARGET_ACTION, targeted)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i)
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[eps].append(overall_reward / num_episodes)

In [ ]:
def plot_norms(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    x, ys = np.array(list(episode_durations.keys())), np.array(list(episode_durations.values()))
    
    plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
    plt.xlabel('L2 Norm')
    plt.ylabel('Average Reward')
    
    mu = np.mean(ys, axis=1)
    plt.plot(x, mu)
    stds = np.std(ys, axis = 1)
    plt.fill_between(x, mu + stds , mu - stds, alpha=0.2)
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [ ]:
episodes = {}
for l2norm in np.arange(0,0.51,0.05):
    episodes[np.round(l2norm, 2)] = []

fgsm_t = {}
fgsm_ut = {}
for eps in np.arange(0.0, 0.201, 0.02):
    fgsm_t[eps] = []
    fgsm_ut[eps] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

"""
i = 0
while i < 12:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_model(agent, episodes)
        print(f"{i} noise: {episodes}")
        i += 1

print("---")
print(f"noise: {episodes}")
"""

i = 0
while i < 12:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        apply_fgsm(agent, fgsm_t, False)
        print(f"{i} fgsm (ut): {fgsm_t}")
        i += 1

print("---")
print(f"fgsm (ut): {fgsm_t}")

0 fgsm (ut): {0.0: [92.17299999998518], 0.02: [60.36899999997546], 0.04: [13.333000000012724], 0.06: [-50.00000000000659], 0.08: [-50.00000000000659], 0.1: [-50.00000000000659], 0.12: [-50.00000000000659], 0.14: [-50.00000000000659], 0.16: [-50.00000000000659], 0.18: [-48.60100000000059], 0.2: [-44.30699999999133]}
1 fgsm (ut): {0.0: [92.17299999998518, 70.57399999997457], 0.02: [60.36899999997546, 21.8830000000078], 0.04: [13.333000000012724, 6.243999999995066], 0.06: [-50.00000000000659, -28.633999999977874], 0.08: [-50.00000000000659, -28.278999999974662], 0.1: [-50.00000000000659, -38.5059999999713], 0.12: [-50.00000000000659, -33.886999999974286], 0.14: [-50.00000000000659, -48.629000000001824], 0.16: [-50.00000000000659, -38.47499999996869], 0.18: [-48.60100000000059, -37.377999999972545], 0.2: [-44.30699999999133, -28.966999999982335]}
2 fgsm (ut): {0.0: [92.17299999998518, 70.57399999997457, 96.36599999999328], 0.02: [60.36899999997546, 21.8830000000078, -22.09799999997513], 0.

In [ ]:
print("noise: {0.0: [91.99899999998568, 77.22399999997548, 96.52599999999386, 81.07099999998191, 95.4109999999937, 95.17999999999294, 91.6919999999895, 93.87099999998826, 92.09399999998604, 86.11799999998317, 96.31299999999345, 96.23999999999428], 0.05: [37.16000000001536, 20.603000000015463, 18.194000000012355, -39.608999999971935, 35.97600000000711, 5.058999999995617, -22.95299999998041, 68.4659999999969, -26.442999999982217, 6.404999999995921, 22.396000000012542, 4.255999999996703], 0.1: [-5.401000000003332, 6.50999999999612, 2.8829999999953757, -31.513999999974708, 41.690000000015374, -26.990999999975493, -16.182999999979767, -5.439000000006087, -17.331999999988465, 1.4960000000002833, 20.123000000006083, -26.893999999973936], 0.15: [-29.912999999978034, -12.932999999999748, -17.77699999998073, -30.54899999997725, 35.14500000001401, -16.973999999994696, -36.08799999997321, -15.498999999988694, -20.36899999997934, -18.62299999998387, 6.319999999994682, -47.39700000000098], 0.2: [-23.121999999983345, -5.837000000006614, -24.594999999973233, -16.892999999988486, 9.048999999994901, -15.846999999997728, -35.50799999997498, -38.54399999997051, -26.57999999997928, -21.177999999986707, -13.140999999986866, -48.835000000005984], 0.25: [-33.12599999997467, -7.5360000000030665, -26.471999999975964, -8.957000000001235, 15.405000000010345, -30.56899999997383, -29.790999999971355, -32.163999999977364, -22.25999999998497, -32.60199999997338, -11.998000000001882, -45.03499999998488], 0.3: [-31.97399999997395, -15.52399999998036, -30.677999999973494, -9.69100000000424, 14.53699999999768, -33.87399999997151, -30.032999999976433, -31.476999999974183, -29.745999999978935, -24.592999999983572, -18.77899999998899, -43.88699999998207], 0.35: [-33.373999999972824, -10.569999999999864, -33.21099999997211, -9.860000000003375, 4.875999999992762, -40.1499999999683, -20.192999999984217, -30.881999999975065, -26.625999999971704, -24.94599999997737, -30.4019999999735, -42.90299999997959], 0.4: [-37.5049999999698, -19.33799999997814, -37.57699999997148, -3.0380000000065723, 4.78599999999411, -35.95199999997119, -14.429999999986107, -25.786999999981322, -28.67999999997575, -22.329999999988853, -26.39399999998108, -37.16299999997353], 0.45: [-30.270999999973842, -24.92799999997464, -34.827999999971546, -13.409999999984501, -7.592000000002323, -35.55099999997203, -28.464999999972733, -19.51799999998508, -29.32099999997269, -31.818999999974547, -27.433999999975775, -33.15199999997593], 0.5: [-29.959999999974183, -33.86599999997234, -36.202999999969855, -12.098000000004943, -7.8460000000076935, -28.02299999997342, -17.140999999988193, -38.230999999972326, -32.01999999997295, -28.391999999979575, -40.4989999999695, -27.006999999979854]}")
print("fgsm (t): {0.0: [92.12199999998683, 78.16699999998752, 96.48699999999411, 79.81799999999076, 95.43199999999182, 96.12199999999217, 91.69799999998948, 94.52799999999057, 90.85899999998692, 72.59099999998104, 96.3229999999931, 95.79199999999395], 0.02: [60.49499999998429, 0.8859999999974495, -9.94100000000339, -27.829999999977687, 36.217000000014, -5.314999999997675, -39.991999999968215, 35.163000000015245, 72.72599999999149, -11.390000000000295, 65.58199999998877, 55.52099999999749], 0.04: [17.068000000002577, -12.415999999994352, -30.221999999974813, -50.00000000000659, 74.97699999997747, -10.982000000005103, -46.019999999990404, 46.05300000000563, 51.00899999999273, 8.4149999999974, 70.60499999998956, 52.80899999999392], 0.06: [-31.771999999977194, -23.142999999979235, -38.719999999971044, -50.00000000000659, 73.55299999999076, 21.324000000014465, -46.3019999999904, 9.025000000001498, 44.93000000000962, 0.7820000000001557, 59.22199999998479, 7.056999999997053], 0.08: [-40.211999999968924, -34.23799999997208, -41.653999999972754, -50.00000000000659, 92.94799999999427, 25.52500000000944, -43.375999999978795, 12.183999999999417, -2.576000000004243, -10.608000000001102, 62.48599999998814, -14.657999999991823], 0.1: [-40.187999999971865, -47.137999999994356, -48.64500000000075, -44.60199999999002, 95.46399999999286, 22.095000000006348, -44.84999999998876, 9.043999999997483, -21.332999999988665, 25.412000000017247, 24.939000000015568, -48.5230000000003], 0.12: [-35.719999999969374, -48.53500000000489, -46.058999999994064, -46.08299999999415, 68.03899999998575, 22.59000000001412, -38.33399999997018, -28.481999999976487, -32.976999999973685, 9.795999999999763, 32.65800000001497, -47.16199999999899], 0.14: [-44.26699999998436, -45.64599999998999, -50.00000000000659, -48.675000000001994, 32.54400000001628, 25.302000000005158, -28.113999999975366, -35.86099999997237, -32.18999999997213, 14.10500000000757, 19.552000000011045, -47.62499999999698], 0.16: [-47.18099999999906, -50.00000000000659, -42.94399999997637, -48.705000000000965, 15.66500000000965, -8.36200000000708, -25.65399999997555, -48.665000000000816, -42.993999999982805, 0.38799999999673396, 23.350000000017047, -47.1889999999954], 0.18: [-43.27499999997843, -45.607999999987875, -42.99399999997968, -47.48999999999563, -2.0950000000071745, -14.463999999986598, -26.80999999998066, -48.654000000005325, -38.77699999996841, -22.54299999998526, 13.96300000000389, -43.15799999997828], 0.2: [-45.91999999998902, -48.56500000000159, -50.00000000000659, -33.081999999977, -14.661999999994087, -27.573999999975207, -23.894999999982787, -50.00000000000659, -41.33399999997051, -32.03699999997503, -5.611000000002257, -47.49099999999564]}")

noise: {0.0: [91.99899999998568, 77.22399999997548, 96.52599999999386, 81.07099999998191, 95.4109999999937, 95.17999999999294, 91.6919999999895, 93.87099999998826, 92.09399999998604, 86.11799999998317, 96.31299999999345, 96.23999999999428], 0.05: [37.16000000001536, 20.603000000015463, 18.194000000012355, -39.608999999971935, 35.97600000000711, 5.058999999995617, -22.95299999998041, 68.4659999999969, -26.442999999982217, 6.404999999995921, 22.396000000012542, 4.255999999996703], 0.1: [-5.401000000003332, 6.50999999999612, 2.8829999999953757, -31.513999999974708, 41.690000000015374, -26.990999999975493, -16.182999999979767, -5.439000000006087, -17.331999999988465, 1.4960000000002833, 20.123000000006083, -26.893999999973936], 0.15: [-29.912999999978034, -12.932999999999748, -17.77699999998073, -30.54899999997725, 35.14500000001401, -16.973999999994696, -36.08799999997321, -15.498999999988694, -20.36899999997934, -18.62299999998387, 6.319999999994682, -47.39700000000098], 0.2: [-23.121999

In [ ]:
def eval_scale(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for scale in np.arange(1.0,7.01,0.5):
        env = NormalizedEnv(PointPushEnv(scale))

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(scale, 2)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for scale in np.arange(1.0,7.01,0.5):
    episodes[np.round(scale, 2)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 12:
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_scale(agent, episodes)
        print(f"{i} scale: {episodes}")
        i += 1

print("---")
print(f"scale: {episodes}")

0 scale: {1.0: [-16.57799999999597], 1.5: [-19.216999999988637], 2.0: [1.1950000000009502], 2.5: [79.61899999998042], 3.0: [80.2199999999789], 3.5: [89.51399999998323], 4.0: [92.54299999998803], 4.5: [93.1709999999893], 5.0: [93.57299999998813], 5.5: [87.02199999998922], 6.0: [52.64399999999542], 6.5: [22.738000000016577], 7.0: [-9.231999999999802]}
1 scale: {1.0: [-16.57799999999597, -50.00000000000659], 1.5: [-19.216999999988637, -50.00000000000659], 2.0: [1.1950000000009502, 74.27999999998768], 2.5: [79.61899999998042, 70.84799999998026], 3.0: [80.2199999999789, 88.49899999998473], 3.5: [89.51399999998323, 95.9829999999925], 4.0: [92.54299999998803, 70.22999999997117], 4.5: [93.1709999999893, 34.70400000001065], 5.0: [93.57299999998813, -4.062000000003785], 5.5: [87.02199999998922, -39.32499999997218], 6.0: [52.64399999999542, -47.19799999999457], 6.5: [22.738000000016577, -50.00000000000659], 7.0: [-9.231999999999802, -50.00000000000659]}
2 scale: {1.0: [-16.57799999999597, -50.000

In [ ]:
def eval_starting_position(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for extra_range in np.arange(0.0, 0.401, 0.05):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            extra = np.random.uniform(-0.1 - extra_range, 0.1 + extra_range, env.starting_point.shape)
            #extra = np.random.uniform(0.1, 0.1 + extra_range, env.starting_point.shape)
            #extra = extra * (2*np.random.randint(0,2,size=env.starting_point.shape)-1)
            env.unwrapped.state = np.array(env.starting_point + extra, dtype=np.float32)
            env.unwrapped.state[2] += math.pi / 2. # start facing up
            env.unwrapped.state[2] = env.state[2] % (2 * math.pi)
            observation = env.normalised_state()

            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(extra_range, 3)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for extra_range in np.arange(0.0, 0.401, 0.05):
    episodes[np.round(extra_range, 3)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 12:
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_starting_position(agent, episodes)
        print(f"{i} extra_range: {episodes}")
        i += 1

print("---")
print(f"extra_range: {episodes}")

0 extra_range: {0.0: [92.41099999998673], 0.05: [92.39799999998621], 0.1: [90.09599999998632], 0.15: [81.38699999998663], 0.2: [67.82199999998174], 0.25: [57.964999999996046], 0.3: [39.54700000001472], 0.35: [15.490000000004795], 0.4: [14.154000000002622]}
1 extra_range: {0.0: [92.41099999998673, 75.31599999997331], 0.05: [92.39799999998621, 65.712999999978], 0.1: [90.09599999998632, 52.33400000000038], 0.15: [81.38699999998663, 38.81300000001639], 0.2: [67.82199999998174, 42.34200000001039], 0.25: [57.964999999996046, 22.35300000000981], 0.3: [39.54700000001472, 25.764000000013713], 0.35: [15.490000000004795, 2.193999999997165], 0.4: [14.154000000002622, 13.460000000001083]}
2 extra_range: {0.0: [92.41099999998673, 75.31599999997331, 96.67499999999403], 0.05: [92.39799999998621, 65.712999999978, 82.71099999998486], 0.1: [90.09599999998632, 52.33400000000038, 61.4769999999846], 0.15: [81.38699999998663, 38.81300000001639, 61.09099999998176], 0.2: [67.82199999998174, 42.34200000001039, 

In [18]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def save_trajectories(agent, episode_durations, dirty):
    agent.eval()

    max_episode_length = 500
    agent.is_training = False

    num_episodes = 10

    l2norm = 0.3
    episode_durations.append([])
    
    for i_episode in range(num_episodes):
        path = {"overall_reward": 0, "worker": []}

        observation = env.reset()

        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_steps = 0
        overall_reward = 0
        done = False
        while not done:
            if dirty:
                state = state + state_range * torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

            action = agent.select_action(state, False, False)
            action_i = action.detach().cpu().squeeze(0).numpy()
            path["worker"].append((episode_steps, state_.detach().cpu().squeeze(0).numpy(), action.detach().cpu().squeeze(0).numpy()))

            observation, reward, done, info = env.step(action_i) 
            
            if max_episode_length and episode_steps >= max_episode_length - 1:
                done = True
            
            episode_steps += 1 

            overall_reward += reward

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        path["overall_reward"] = overall_reward
        episode_durations[-1].append(path)

In [22]:
episodes = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 12:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        save_trajectories(agent, episodes, False)
        #print(f"{i} paths: {episodes}")
        i += 1

print("----")
#print(f"paths: {episodes}")

episodes.pop(1)
episodes.pop(3 - 1)

torch.save(episodes, "TD3_PointPush_clean_eps.pt")

----
